# **Sandalwood cultivation Predictor**



---



**Importing the Dependencies.**

In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler



---



**Data Collection , Analysis & Cleaning**

In [35]:
df = pd.read_csv('/content/sample_data/GfG (2).csv')

In [36]:
df.head()

,Unnamed: 0,id,soil_type,soil_ph,iron_content,phosphorous_content,Potassium_content,Sowing Thickness,Temperature,Rainfall,Humidity,Fertilizer_freq,Fertilizer_Date,Disease,sec_para,Target
0,0,568d39f2d8654ff992e45c1393938994,loamy sand,8.373032,Mid,Low,Low,75,39,1426,36,4,2016-07-04 05:26:13,Seedling,NaN,0
1,1,7c1a6e681aae44f49b9bbeab063aa527,loamy sand,8.010889,High,Very High,Low,124,40,975,26,2,2016-06-11 20:19:49,No Disease,NaN,0
2,2,ad4f1ec2ce0a46c18c738141da594e8a,loamy sand,8.418084,Mid,Very High,High,127,32,779,35,2,2016-08-29 23:58:17,Spike,NaN,0
3,3,9069e2cecc074820a3a32438b93dd6b2,clay rich soil,6.020134,High,Mid,High,75,26,768,36,4,2015-06-22 22:02:55,Spike,NaN,0
4,4,31afa45fd3fe477583078fd520047276,clay rich soil,8.084770,Mid,Mid,Very Low,50,38,1101,61,4,2017-08-11 16:06:57,Spike,NaN,0


In [37]:
df.shape

(200000, 16)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           200000 non-null  int64  
 1   id                   200000 non-null  object 
 2   soil_type            200000 non-null  object 
 3   soil_ph              200000 non-null  float64
 4   iron_content         200000 non-null  object 
 5   phosphorous_content  200000 non-null  object 
 6   Potassium_content    200000 non-null  object 
 7   Sowing Thickness     200000 non-null  int64  
 8   Temperature          200000 non-null  int64  
 9   Rainfall             200000 non-null  int64  
 10  Humidity             200000 non-null  int64  
 11  Fertilizer_freq      200000 non-null  int64  
 12  Fertilizer_Date      200000 non-null  object 
 13  Disease              200000 non-null  object 
 14  sec_para             0 non-null       float64
 15  Target           

In [39]:
df.isnull().sum()

Unnamed: 0                  0
id                          0
soil_type                   0
soil_ph                     0
iron_content                0
phosphorous_content         0
Potassium_content           0
Sowing Thickness            0
Temperature                 0
Rainfall                    0
Humidity                    0
Fertilizer_freq             0
Fertilizer_Date             0
Disease                     0
sec_para               200000
Target                      0
dtype: int64

In [40]:
df.drop(['Unnamed: 0'], axis=1,inplace=True)

In [41]:
df.isnull().sum()

id                          0
soil_type                   0
soil_ph                     0
iron_content                0
phosphorous_content         0
Potassium_content           0
Sowing Thickness            0
Temperature                 0
Rainfall                    0
Humidity                    0
Fertilizer_freq             0
Fertilizer_Date             0
Disease                     0
sec_para               200000
Target                      0
dtype: int64

In [42]:
df.corr(method='pearson')

,soil_ph,Sowing Thickness,Temperature,Rainfall,Humidity,Fertilizer_freq,sec_para,Target
soil_ph,1.000000,-0.003771,0.002904,-0.002130,0.001527,-0.004578,NaN,0.000670
Sowing Thickness,-0.003771,1.000000,0.002041,0.004022,0.001439,0.003543,NaN,-0.005467
Temperature,0.002904,0.002041,1.000000,0.000850,0.000577,0.002267,NaN,-0.022002
Rainfall,-0.002130,0.004022,0.000850,1.000000,0.003130,0.002097,NaN,0.001031
Humidity,0.001527,0.001439,0.000577,0.003130,1.000000,0.001073,NaN,-0.005733
Fertilizer_freq,-0.004578,0.003543,0.002267,0.002097,0.001073,1.000000,NaN,0.007102
sec_para,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Target,0.000670,-0.005467,-0.022002,0.001031,-0.005733,0.007102,NaN,1.000000


In [43]:
df.corr().style.background_gradient(cmap="Blues")

/usr/local/lib/python3.9/dist-packages/pandas/io/formats/style.py:3555: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/usr/local/lib/python3.9/dist-packages/pandas/io/formats/style.py:3556: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,soil_ph,Sowing Thickness,Temperature,Rainfall,Humidity,Fertilizer_freq,sec_para,Target
soil_ph,1.000000,-0.003771,0.002904,-0.002130,0.001527,-0.004578,nan,0.000670
Sowing Thickness,-0.003771,1.000000,0.002041,0.004022,0.001439,0.003543,nan,-0.005467
Temperature,0.002904,0.002041,1.000000,0.000850,0.000577,0.002267,nan,-0.022002
Rainfall,-0.002130,0.004022,0.000850,1.000000,0.003130,0.002097,nan,0.001031
Humidity,0.001527,0.001439,0.000577,0.003130,1.000000,0.001073,nan,-0.005733
Fertilizer_freq,-0.004578,0.003543,0.002267,0.002097,0.001073,1.000000,nan,0.007102
sec_para,nan,nan,nan,nan,nan,nan,nan,nan
Target,0.000670,-0.005467,-0.022002,0.001031,-0.005733,0.007102,nan,1.000000


In [44]:
df.describe()

,soil_ph,Sowing Thickness,Temperature,Rainfall,Humidity,Fertilizer_freq,sec_para,Target
count,200000.000000,200000.000000,200000.000000,200000.00000,200000.000000,200000.000000,0.0,200000.000000
mean,6.744437,99.406405,29.486730,949.05114,56.934315,1.995520,NaN,0.203140
std,1.009953,34.635083,8.649069,317.42102,18.771617,1.413973,NaN,0.402337
min,5.000029,40.000000,15.000000,400.00000,25.000000,0.000000,NaN,0.000000
25%,5.865526,69.000000,22.000000,674.00000,41.000000,1.000000,NaN,0.000000
50%,6.743131,99.000000,29.000000,950.00000,57.000000,2.000000,NaN,0.000000
75%,7.615492,129.000000,37.000000,1223.00000,73.000000,3.000000,NaN,0.000000
max,8.499999,159.000000,44.000000,1499.00000,89.000000,4.000000,NaN,1.000000


In [45]:
df['Target'].value_counts()

0    159372
1     40628
Name: Target, dtype: int64

In [46]:
df.groupby('Target').mean()

,soil_ph,Sowing Thickness,Temperature,Rainfall,Humidity,Fertilizer_freq,sec_para
Target,,,,,,,
0,6.744095,99.502002,29.582813,948.885952,56.988649,1.990450,NaN
1,6.745776,99.031407,29.109826,949.699124,56.721178,2.015408,NaN


In [47]:
df.groupby('Disease').mean()

,soil_ph,Sowing Thickness,Temperature,Rainfall,Humidity,Fertilizer_freq,sec_para,Target
Disease,,,,,,,,
No Disease,6.747135,99.499596,29.456329,947.963145,56.949335,1.999626,NaN,0.607204
Seedling,6.744408,99.488825,29.524970,949.357666,56.897184,1.997123,NaN,0.000000
Spike,6.741759,99.230844,29.479157,949.837532,56.956215,1.989804,NaN,0.000000


In [48]:
df1 = df[df['Disease']=="No Disease"]

In [49]:
df1.groupby('Target').mean()

,soil_ph,Sowing Thickness,Temperature,Rainfall,Humidity,Fertilizer_freq,sec_para
Target,,,,,,,
0,6.749236,100.223347,29.991972,945.279583,57.302032,1.975230,NaN
1,6.745776,99.031407,29.109826,949.699124,56.721178,2.015408,NaN


In [50]:
df1.describe()

,soil_ph,Sowing Thickness,Temperature,Rainfall,Humidity,Fertilizer_freq,sec_para,Target
count,66910.000000,66910.000000,66910.000000,66910.000000,66910.000000,66910.000000,0.0,66910.000000
mean,6.747135,99.499596,29.456329,947.963145,56.949335,1.999626,NaN,0.607204
std,1.009332,34.599904,8.642117,317.235139,18.774506,1.413373,NaN,0.488376
min,5.000029,40.000000,15.000000,400.000000,25.000000,0.000000,NaN,0.000000
25%,5.866284,70.000000,22.000000,673.000000,41.000000,1.000000,NaN,0.000000
50%,6.747975,100.000000,29.000000,949.000000,57.000000,2.000000,NaN,1.000000
75%,7.621307,129.000000,37.000000,1222.000000,73.000000,3.000000,NaN,1.000000
max,8.499965,159.000000,44.000000,1499.000000,89.000000,4.000000,NaN,1.000000


In [51]:
df1['Target'].value_counts()

1    40628
0    26282
Name: Target, dtype: int64

In [52]:
df = df1

In [53]:
df.drop("Disease", axis=1, inplace=True)

<ipython-input-53-db5fc6dfc1db>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop("Disease", axis=1, inplace=True)


In [54]:
df.describe()

,soil_ph,Sowing Thickness,Temperature,Rainfall,Humidity,Fertilizer_freq,sec_para,Target
count,66910.000000,66910.000000,66910.000000,66910.000000,66910.000000,66910.000000,0.0,66910.000000
mean,6.747135,99.499596,29.456329,947.963145,56.949335,1.999626,NaN,0.607204
std,1.009332,34.599904,8.642117,317.235139,18.774506,1.413373,NaN,0.488376
min,5.000029,40.000000,15.000000,400.000000,25.000000,0.000000,NaN,0.000000
25%,5.866284,70.000000,22.000000,673.000000,41.000000,1.000000,NaN,0.000000
50%,6.747975,100.000000,29.000000,949.000000,57.000000,2.000000,NaN,1.000000
75%,7.621307,129.000000,37.000000,1222.000000,73.000000,3.000000,NaN,1.000000
max,8.499965,159.000000,44.000000,1499.000000,89.000000,4.000000,NaN,1.000000


In [55]:
df.corr()

,soil_ph,Sowing Thickness,Temperature,Rainfall,Humidity,Fertilizer_freq,sec_para,Target
soil_ph,1.000000,-0.003723,0.001439,-0.001628,0.000536,-0.005994,NaN,-0.001674
Sowing Thickness,-0.003723,1.000000,0.000139,0.005922,0.007524,0.000209,NaN,-0.016824
Temperature,0.001439,0.000139,1.000000,0.002188,0.001085,0.004016,NaN,-0.049851
Rainfall,-0.001628,0.005922,0.002188,1.000000,0.005029,0.002942,NaN,0.006804
Humidity,0.000536,0.007524,0.001085,0.005029,1.000000,-0.002738,NaN,-0.015110
Fertilizer_freq,-0.005994,0.000209,0.004016,0.002942,-0.002738,1.000000,NaN,0.013883
sec_para,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Target,-0.001674,-0.016824,-0.049851,0.006804,-0.015110,0.013883,NaN,1.000000




---



**Label Encoding**

In [56]:
le = LabelEncoder()
label1 = le.fit_transform(df['Temperature'])
label2 = le.fit_transform(df['Rainfall'])
label3 = le.fit_transform(df['Humidity'])
label4 = le.fit_transform(df['iron_content'])
label5 = le.fit_transform(df['phosphorous_content'])
label6 = le.fit_transform(df['Potassium_content'])
label7 = le.fit_transform(df['soil_type'])

In [57]:
df.drop("Temperature", axis=1, inplace=True)
df["Temperature"] = label1

df.drop("Rainfall", axis=1, inplace=True)
df["Rainfall"] = label2

df.drop("Humidity", axis=1, inplace=True)
df["Humidity"] = label3

df.drop("iron_content", axis=1, inplace=True)
df["iron_content"] = label4

df.drop("phosphorous_content", axis=1, inplace=True)
df["phosphorous_content"] = label5

df.drop("Potassium_content", axis=1, inplace=True)
df["Potassium_content"] = label6

df.drop("soil_type", axis=1, inplace=True)
df["soil_type"] = label7

In [58]:
df

,id,soil_ph,Sowing Thickness,Fertilizer_freq,Fertilizer_Date,sec_para,Target,Temperature,Rainfall,Humidity,iron_content,phosphorous_content,Potassium_content,soil_type
1,7c1a6e681aae44f49b9bbeab063aa527,8.010889,124,2,2016-06-11 20:19:49,NaN,0,25,575,1,0,3,1,1
6,f7749fd33a3d4d2ca6b376bc3b2029bd,6.121119,107,3,2015-04-12 14:50:05,NaN,1,11,884,55,0,1,4,0
7,38b9da19a38240cc9e9b33746c186b46,5.786099,88,0,2016-10-05 11:50:03,NaN,1,5,544,15,0,1,3,1
9,979100635c1e4a1aaa67e39924248182,7.555483,56,1,2017-01-25 12:02:00,NaN,0,1,1084,49,3,1,1,2
13,bc819149e2644759ba4e8cd834e7e477,6.310864,93,3,2016-02-22 17:35:10,NaN,1,16,883,7,3,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199993,0c378470462640ea9c9d008aaa05cb2c,7.636906,157,2,2016-10-28 12:13:02,NaN,1,12,1078,6,2,2,0,1
199994,49201d0abbbd4a6bb1ad1a7ab32a542e,7.116873,90,0,2015-12-19 02:47:59,NaN,1,18,570,59,2,0,3,2
199996,b621b1898a9448eb9d88a61880767743,7.897183,119,4,2015-07-04 12:47:04,NaN,1,24,237,17,0,4,2,3
199997,316fb6165b8b41b39a564809039580f6,7.249173,158,0,2017-09-04 17:42:14,NaN,1,7,464,23,2,2,1,2


In [59]:
df.drop("sec_para", axis=1, inplace=True)

In [60]:
df.drop("id", axis=1, inplace=True)

In [61]:
df.drop("Fertilizer_Date", axis=1, inplace=True)

In [62]:
df

,soil_ph,Sowing Thickness,Fertilizer_freq,Target,Temperature,Rainfall,Humidity,iron_content,phosphorous_content,Potassium_content,soil_type
1,8.010889,124,2,0,25,575,1,0,3,1,1
6,6.121119,107,3,1,11,884,55,0,1,4,0
7,5.786099,88,0,1,5,544,15,0,1,3,1
9,7.555483,56,1,0,1,1084,49,3,1,1,2
13,6.310864,93,3,1,16,883,7,3,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...
199993,7.636906,157,2,1,12,1078,6,2,2,0,1
199994,7.116873,90,0,1,18,570,59,2,0,3,2
199996,7.897183,119,4,1,24,237,17,0,4,2,3
199997,7.249173,158,0,1,7,464,23,2,2,1,2




---



In [63]:
df['target']=df['Target']

In [64]:
df.drop("Target", axis=1, inplace=True)

In [65]:
df

,soil_ph,Sowing Thickness,Fertilizer_freq,Temperature,Rainfall,Humidity,iron_content,phosphorous_content,Potassium_content,soil_type,target
1,8.010889,124,2,25,575,1,0,3,1,1,0
6,6.121119,107,3,11,884,55,0,1,4,0,1
7,5.786099,88,0,5,544,15,0,1,3,1,1
9,7.555483,56,1,1,1084,49,3,1,1,2,0
13,6.310864,93,3,16,883,7,3,4,4,4,1
...,...,...,...,...,...,...,...,...,...,...,...
199993,7.636906,157,2,12,1078,6,2,2,0,1,1
199994,7.116873,90,0,18,570,59,2,0,3,2,1
199996,7.897183,119,4,24,237,17,0,4,2,3,1
199997,7.249173,158,0,7,464,23,2,2,1,2,1




---



**Splitting the data to training data & Test data**

In [66]:
X = df.iloc[ 0:66911, 0:10]
Y = df['target']

In [67]:
print(X)

         soil_ph  Sowing Thickness  Fertilizer_freq  Temperature  Rainfall  \
1       8.010889               124                2           25       575   
6       6.121119               107                3           11       884   
7       5.786099                88                0            5       544   
9       7.555483                56                1            1      1084   
13      6.310864                93                3           16       883   
...          ...               ...              ...          ...       ...   
199993  7.636906               157                2           12      1078   
199994  7.116873                90                0           18       570   
199996  7.897183               119                4           24       237   
199997  7.249173               158                0            7       464   
199999  8.259262               120                3            3       946   

        Humidity  iron_content  phosphorous_content  Potassium_

In [68]:
print(Y)

1         0
6         1
7         1
9         0
13        1
         ..
199993    1
199994    1
199996    1
199997    1
199999    1
Name: target, Length: 66910, dtype: int64


In [69]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [70]:
print(X.shape, X_train.shape, X_test.shape)

(66910, 10) (53528, 10) (13382, 10)




---



**Data Normalization.**

In [71]:
scaler = StandardScaler()
scaler.fit(X)
scaled = scaler.transform(X)



---



**Model Training.**

In [95]:
# machine learning model_pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier 
from sklearn. tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn. naive_bayes import GaussianNB
model_pipeline = []
model_pipeline.append(LogisticRegression (solver='liblinear'))
model_pipeline.append(SVC())
model_pipeline.append(KNeighborsClassifier())
model_pipeline.append(DecisionTreeClassifier())
model_pipeline.append(RandomForestClassifier())
model_pipeline.append(GaussianNB())

In [99]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
model_list = ['Logistic Regression', 'SVM', 'KNN', 'Decision Tree', 'Random Forest', 'Naive Bayes'] 
acc_list = []
auc_list = []
cm_list = []
for model in model_pipeline:
    model.fit(X_train, Y_train)
    Y_pred =model.predict(X_test)
    acc_list.append(metrics.accuracy_score (Y_test, Y_pred))
    fpr, tpr, thresholds = metrics.roc_curve (Y_test, Y_pred) 
    auc_list.append(round (metrics. auc (fpr, tpr), 2)) 
    cm_list.append(confusion_matrix(Y_test, Y_pred))

In [100]:
result_df = pd.DataFrame({ 'Model' :model_list, 'Accuracy': acc_list, 'AUC': auc_list}) 
result_df

,Model,Accuracy,AUC
0,Logistic Regression,0.628082,0.55
1,SVM,0.604095,0.50
2,KNN,0.565087,0.52
3,Decision Tree,0.928561,0.92
4,Random Forest,0.947243,0.95
5,Naive Bayes,0.683829,0.62


Hence,Random Forest to be selected as it has highest accuracy.